<a href="https://colab.research.google.com/github/Requenamar3/Data-Mining/blob/main/Data_Mining_Module1_Ecommerce_CLV%20filtered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Customer Lifetime Value model

In [1]:
#package used for customer lifetime value modeling
!pip install lifetimes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00


In [2]:
 # installing the pandas_profiling package for data analysis and generating statistical report summaries.
!pip install ydata_profiling


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 37.8 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=54a6333413103546c777d277e847879a44b529b14ebcb076e0abe966021da9dc
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling 

In [3]:
# Customer Lifetime Value (CLV or CLTV)
# Buy till you die... (BG/NBD)

# Import the lifetimes package for customer lifetime value analysis.
import lifetimes
# Import pandas for data manipulation and analysis.
import pandas as pd
# Import numpy for numerical computing.
import numpy as np
# Import datetime for handling dates and times.
import datetime as dt
# Import matplotlib.pyplot for creating static, animated, and interactive visualizations.
import matplotlib.pyplot as plt
# Import seaborn for making statistical graphics.
import seaborn as sns
# Import BetaGeoFitter from lifetimes for fitting the BG/NBD model.
from lifetimes import BetaGeoFitter
# Import GammaGammaFitter from lifetimes for fitting the Gamma-Gamma model.
from lifetimes import GammaGammaFitter
# Import plot_frequency_recency_matrix for visualizing the relationship between frequency and recency.
from lifetimes.plotting import plot_frequency_recency_matrix

# Set pandas display options to improve the readability of output.
# Set the maximum number of rows to display to 500.
pd.set_option('display.max_rows', 500)
# Set the maximum number of columns to display to 500.
pd.set_option('display.max_columns', 500)
# Set the width of the display in characters to 1000.
pd.set_option('display.width', 1000)


In [37]:
#Import the ProfileReport .For creating comprehensive exploratory data analysis reports.
from ydata_profiling import ProfileReport

In [38]:
EC= pd.read_csv("https://raw.githubusercontent.com/Requenamar3/Data-Mining/main/sales_2023-10-01_2024-03-06.csv")

In [39]:
EC.head()

month    financial_status order_name              customer_email    customer_id customer_type variant_sku  product_price                            product_title shipping_region shipping_postal_code customer_cohort_week customer_cohort_quarter customer_cohort_month product_type  total_sales  net_sales  total_cost  units_per_transaction
0  2023-10                paid    #805723      mayapriebe33@gmail.com  6616065245299     Returning   SQ9178530           0.00  Bloomsy Original (Ships every 1 Months)        Colorado                80525             2023-W19                 2023-04               2023-05          NaN         0.00       0.00         0.0                      1
1  2023-10  partially_refunded    #810617  vincent.nicolais@yahoo.com  6261751251059     Returning         NaN           0.00                                      NaN         Georgia                31820             2022-W41                 2022-10               2022-10          NaN        83.88       0.00         0.0                      0
2  2023-10  partially_refunded    #810617  vincent.nicolais@yahoo.com  6261751251059     Returning   SQ8862610         642.99                          Bloomsy Premium         Georgia                31820             2022-W41                 2022-10               2022-10          NaN       694.43     642.99         0.0                      1
3  2023-10  partially_refunded    #810431    cjsherrillmain@gmail.com  3496267448435     Returning   SQ1726938          69.99                           Bloomsy Deluxe        Maryland                20636             2020-W48                 2020-10               2020-11          NaN        74.19      69.99         0.0                      1
4  2023-10  partially_refunded    #810431    cjsherrillmain@gmail.com  3496267448435     Returning         NaN           0.00                                      NaN        Maryland                20636             2020-W48                 2020-10               2020-11          NaN       -10.60     -10.60         0.0                      0

In [40]:
# Select only the desired columns
EC_filtered = ['customer_id', 'order_name','variant_sku','month', 'customer_cohort_month', 'total_sales', 'units_per_transaction']
EC= EC[EC_filtered]
EC.head()


customer_id order_name variant_sku    month customer_cohort_month  total_sales  units_per_transaction
0  6616065245299    #805723   SQ9178530  2023-10               2023-05         0.00                      1
1  6261751251059    #810617         NaN  2023-10               2022-10        83.88                      0
2  6261751251059    #810617   SQ8862610  2023-10               2022-10       694.43                      1
3  3496267448435    #810431   SQ1726938  2023-10               2020-11        74.19                      1
4  3496267448435    #810431         NaN  2023-10               2020-11       -10.60                      0

In [41]:
# Create a ProfileReport object
profile = ProfileReport(EC, title="EC CLV", explorative=True)

In [42]:
# display report
profile


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
# Remove rows from 'EC' where 'total_sales' is equal to 0.00
EC = EC[EC['total_sales'] != 0.00]


In [44]:
# Remove records with 0 orders qty
EC = EC[EC['units_per_transaction'] != 0]


In [45]:
EC.head()

customer_id order_name variant_sku    month customer_cohort_month  total_sales  units_per_transaction
2   6261751251059    #810617   SQ8862610  2023-10               2022-10       694.43                      1
3   3496267448435    #810431   SQ1726938  2023-10               2020-11        74.19                      1
8   5601422671987    #802622    BB738-24  2023-10               2022-03        54.49                      1
11  6769237557363    #801731    BB727-26  2023-10               2023-07        64.34                      1
13  6272794919027    #807669    BB944-24  2023-10               2022-10        63.74                      1

In [46]:
EC.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44231 entries, 2 to 158180
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            44231 non-null  int64  
 1   order_name             44231 non-null  object 
 2   variant_sku            44214 non-null  object 
 3   month                  44231 non-null  object 
 4   customer_cohort_month  44231 non-null  object 
 5   total_sales            44231 non-null  float64
 6   units_per_transaction  44231 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 3.7+ MB


In [47]:
EC.isnull().sum()

customer_id               0
order_name                0
variant_sku              17
month                     0
customer_cohort_month     0
total_sales               0
units_per_transaction     0
dtype: int64

In [48]:
EC.dropna(inplace=True)  # inplace=True means we dropped them permanently

In [49]:
EC.isnull().sum()

customer_id              0
order_name               0
variant_sku              0
month                    0
customer_cohort_month    0
total_sales              0
units_per_transaction    0
dtype: int64

In [50]:
EC.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44214 entries, 2 to 158180
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            44214 non-null  int64  
 1   order_name             44214 non-null  object 
 2   variant_sku            44214 non-null  object 
 3   month                  44214 non-null  object 
 4   customer_cohort_month  44214 non-null  object 
 5   total_sales            44214 non-null  float64
 6   units_per_transaction  44214 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 2.7+ MB


In [51]:
EC.describe()

customer_id   total_sales  units_per_transaction
count  4.421400e+04  44214.000000           44214.000000
mean   5.562729e+12    117.220923               1.005406
std    1.682923e+12    149.469024               0.084234
min    1.183560e+12      0.220000               1.000000
25%    5.028425e+12     49.530000               1.000000
50%    6.368478e+12     59.600000               1.000000
75%    6.996533e+12     84.790000               1.000000
max    7.301131e+12   3507.540000               5.000000

***find_boundaries*** function: Calculates upper and lower boundaries (quantiles) for a given variable in the DataFrame.

**capping_outliers function:** Caps outliers in a given variable of the DataFrame by replacing values beyond the upper and lower boundaries with those boundaries.

*These functions are designed to efficiently handle outlier treatment in a DataFrame, making the code concise and easy to understand.*

Outliers are replaced with the upper and lower boundaries calculated based on default quantiles (5th and 95th percentiles).

In [52]:
def find_boundaries(df, variable,q1=0.05,q2=0.95):
    # the boundaries are the quantiles
    lower_boundary = df[variable].quantile(q1) # lower quantile
    upper_boundary = df[variable].quantile(q2) # upper quantile
    return upper_boundary, lower_boundary
def capping_outliers(df,variable):
    upper_boundary,lower_boundary =  find_boundaries(df,variable)
    df[variable] = np.where(df[variable] > upper_boundary, upper_boundary,
                       np.where(df[variable] < lower_boundary, lower_boundary, df[variable]))

In [53]:
# Function to cap outliers in the 'Total_sales'
capping_outliers(EC, 'total_sales')

# Function to cap outliers in the 'units_per_transaction'
capping_outliers(EC, 'units_per_transaction')


In [54]:
EC.describe()

customer_id   total_sales  units_per_transaction
count  4.421400e+04  44214.000000                44214.0
mean   5.562729e+12    109.982135                    1.0
std    1.682923e+12    118.461722                    0.0
min    1.183560e+12     42.790000                    1.0
25%    5.028425e+12     49.530000                    1.0
50%    6.368478e+12     59.600000                    1.0
75%    6.996533e+12     84.790000                    1.0
max    7.301131e+12    498.934000                    1.0

The average of sales ammount per transaction is around 110. The high standard deviation of 118 indicates the sales amounts vary. The minimum sales is 42.49 and the maximum 498.94. A quater of the transactions are relatively small in total sales. The median indicates that most customers are spending around $59.60 per transaction. This is valuable information for understanding customer behavior, planning inventory and supply chain management, or developing marketing and sales strategies aimed at the most common transaction size.

**Customer Behavior**: Most customers are spending around $59.60 per transaction .This can help us making decisions about product pricing, promotions, and product bundles to cater to the spending habits of the majority.

**Marketing Strategies**: Marketing efforts can be tailored to the average customer's spending. If most customers spend around $59.60, marketing can highlight products or deals around this price. Upselling strategies could focus on slightly higher-priced items to increase the average spend.
We could recommend additional purchases of accesories, with the goal of slightly increasing the average spend per customer.

#Recency Frecuency Time

In [55]:
# Calculate the unit price
EC['UnitPrice'] = EC['total_sales'] // EC['units_per_transaction']


In [56]:
EC.columns

Index(['customer_id', 'order_name', 'variant_sku', 'month', 'customer_cohort_month', 'total_sales', 'units_per_transaction', 'UnitPrice'], dtype='object')

Using the lifetime package to analyse historical customer transactions data to predict future purchasing behavior.

In [57]:
# generate a sumary df called clv that agreagates transaction data by customers until 3/5/24
clv = lifetimes.utils.summary_data_from_transaction_data(EC,'customer_id','month','total_sales',observation_period_end='2024-03-05')

The summary_data_from_transaction_data function will calculate:

**Frequency:** The number of repeat purchases made by a customer. This is calculated as the total number of purchases minus one, considering the first purchase as a non-repeat purchase.

**Recency**: The age of a customer when they made their most recent purchase. This is calculated as the time between the first purchase and the last purchase.

**T:** The age of the customer in terms of the analysis time frame, calculated as the time between the first purchase and the end of the observation period.

**Monetary value: **An average value of the customer's transactions, typically calculated excluding the first purchase.

In [58]:
clv.head()

frequency  recency      T  monetary_value
customer_id                                             
1183559942259        2.0     62.0   95.0           85.59
1183569084531        1.0     30.0  125.0           42.79
1183569707123        5.0    152.0  156.0           47.07
1183569805427        0.0      0.0   64.0            0.00
1183579504755        0.0      0.0   64.0            0.00

In [59]:
# let's filter so that we only catch customers who have made more than 1 purchase
clv = clv[clv['frequency']>1]

In [60]:
clv.head()

frequency  recency      T  monetary_value
customer_id                                             
1183559942259        2.0     62.0   95.0           85.59
1183569707123        5.0    152.0  156.0           47.07
1183582322803        4.0    123.0  156.0           53.49
1183584551027        5.0    152.0  156.0           43.79
1183594184819        5.0    152.0  156.0           43.99

**Customer 1183559942259:** Two repeat purchases over 95 months with an average spend of 85.59 per purchase.

**Customer 1183569084531:**One repeat purchase over 125 months, averaging 42.79 spent.

**Customer 1183569070123:** Five repeat purchases, customer for 156 months, spending 47.07 on average.

**Customers 1183569085427 & 1183579504755**:  No repeat purchases; average spend is 0.00.

The presence of customers with repeat transactions suggests successful repeat business.Customers with high average transaction values contribute positively to revenue.Zero repeat purchases for some customers highlight possible retention issues, suggesting a need for improved engagement and retention strategies.

#BG/NBD Model

Initializes and fits the Beta-Geometric/Beta-Binomial (BG/NBD) model to the provided summary statistics. These statistics include the frequency (number of repeat transactions), recency (age at the last transaction), and tenure (age at the end of the observation period) for each customer. The BG/NBD model is a probabilistic model used for predicting customer lifetime value (CLV) and future purchase behavior based on historical transaction data. The penalizer coefficient is specified to prevent overfitting during model fitting.

In [28]:
# Initialize the Beta-Geometric/Beta-Binomial model (BG/NBD) with a penalizer coefficient to avoid overfitting
bgf = BetaGeoFitter(penalizer_coef=0.001)

# Fit the BG/NBD model using the summary statistics: frequency (F), recency (R), and tenure (T)
bgf.fit(
    clv['frequency'],  # Frequency represents the number of repeat transactions
    clv['recency'],    # Recency represents the age of the customer at their last transaction
    clv['T']           # Tenure represents the age of the customer at the end of the observation period
)


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 4404 subjects, a: 0.00, alpha: 322.63, b: 0.00, r: 8.02>

Predicts the expected number of purchases for each customer in the next 6 months using the Beta-Geometric/Beta-Binomial (BG/NBD) model. It calculates these predictions based on the customer's historical repeat transaction behavior, considering their frequency (number of repeat transactions), recency (age at their last transaction), and tenure (age at the end of the observation period). Then, it sorts the customers by their expected purchases in descending order and selects the top 5 customers with the highest expected purchases for further analysis or action.

In [29]:
t = 180  # Define the prediction period of 6 months (180 days)

# Predict future purchases for the next 6 months using the BG/NBD model
clv['expected_purc_6_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(
    t,                      # Prediction period
    clv['frequency'],       # Frequency represents the number of repeat transactions
    clv['recency'],         # Recency represents the age of the customer at their last transaction
    clv['T']                # Tenure represents the age of the customer at the end of the observation period
)

# Sort customers by expected purchases in the next 6 months and display the top 5
clv_top5 = clv.sort_values(by='expected_purc_6_months', ascending=False).head(5)


In [30]:
clv_top5

frequency  recency      T  monetary_value  expected_purc_6_months
customer_id                                                                     
5454865727603        5.0    152.0  156.0         112.384                4.896924
5156199071859        5.0    152.0  156.0          87.088                4.896924
5202694045811        5.0    152.0  156.0          49.150                4.896924
5200096985203        5.0    152.0  156.0          62.988                4.896924
5188042293363        5.0    152.0  156.0          53.490                4.896924

These customers have a consistent purchase pattern: they've all made 5 repeat purchases and their most recent purchase was 152 months after their first one. They've been customers for a total of 156 months, indicating a long-term relationship with the company. Despite the similarities in their purchase frequency and timing, they spend different amounts on average, ranging from about 42 to 60 per transaction. The model predicts that each of these customers will make approximately 4.42 purchases in the upcoming 6 months, a fairly high level of engagement. This uniform prediction suggests that these customers are not only regularly active but are also expected to continue this behavior, making them particularly valuable for future revenue. Their varied monetary values, combined with consistent purchasing behavior, might provide opportunities for targeted marketing strategies to further increase their spend.

#Gamma-Gamma Model

In [ ]:
# Calculate and display the correlation between 'frequency' and 'monetary_value' columns in the 'clv' DataFrame.
clv[['frequency', 'monetary_value']].corr()


frequency  monetary_value
frequency        1.000000       -0.169663
monetary_value  -0.169663        1.000000

The correlation coefficient indicates how two variables move together. A value near 1 means they move in the same direction, near -1 means they move in opposite directions, and around 0 means there's no linear relationship.

The correlation of -0.17 between purchase frequency and average monetary value indicates a minor tendency for customers who purchase more often to spend less per transaction, but this trend is not strongly pronounced and should be interpreted cautiously.

In [62]:
# Initialize GammaGammaFitter with a penalizer to prevent overfitting and fit on frequency and monetary value.
ggf = GammaGammaFitter(penalizer_coef=0.01)  # Create GammaGammaFitter model instance with a specified penalizer.
ggf.fit(clv["frequency"], clv["monetary_value"])  # Fit model to customer data for predicting monetary value.


<lifetimes.GammaGammaFitter: fitted with 4404 subjects, p: 4.36, q: 0.66, v: 4.26>

In [63]:
# Calculate 6-month Customer Lifetime Value (CLV) using the GammaGamma and BetaGeo models, then identify top customers.
# The 'customer_lifetime_value' method from GammaGammaFitter ('ggf') combines both frequency/recency (from 'bgf') and
# monetary value to estimate the CLV for the next 6 months ('time=6'), assuming daily ('freq='D'') transactions.
# A discount rate of 0.01 is applied to account for the time value of money.
clv['six_Months_CLV']=ggf.customer_lifetime_value(bgf,
                                   clv["frequency"],
                                   clv["recency"],
                                   clv["T"],
                                   clv["monetary_value"],
                                   time=6,
                                   freq='D',
                                   discount_rate=0.01)
clv.sort_values('six_Months_CLV',ascending=False).head()

frequency  recency      T  monetary_value  six_Months_CLV
customer_id                                                             
6543258091635        5.0    152.0  156.0        1194.816     5744.056762
6569497985139        2.0    123.0  156.0        1227.343     4654.972431
5419749965939        2.0     91.0   95.0         997.868     4339.157212
5455684370547        2.0     92.0  156.0         803.544     3050.406054
5394003755123        2.0     62.0   95.0         606.174     2639.533688

This code calculates a 6-month predicted CLV for each customer, adds these predictions as a new column in the clv DataFrame, and then displays the top 5 customers with the highest predicted CLV values.

Customer ID 6543258091635: A frequent shopper with the highest average spend and projected purchases, making them the most valuable over the next six months.

Customer ID 6569497985139: Not as frequent in purchases but spends a lot when they do; they're expected to continue to be a valuable customer.

Customer ID 5419794965939: Moderately frequent with a good spending average; they're also projected to continue purchasing regularly.

Customer ID 5455684370547: Similar frequency as others but spends less per transaction; still, they're expected to contribute steadily.

Customer ID 5394003755123: Least amount spent on average but anticipated to buy as often as others; their value is lower but not insignificant.

In [64]:
# Segment customers into 4 categories based on their CLV using quartiles.
clv['Segment'] = pd.qcut(clv['six_Months_CLV'], 4, labels=['Hibernating', 'Need Attention', 'Loyal Customers', 'Champions'])  # Assign segments based on CLV quartiles.


In [34]:
clv.head()

frequency  recency      T  monetary_value  expected_purc_6_months  six_Months_CLV         Segment
customer_id                                                                                                     
1183559942259        2.0     62.0   95.0           85.59                4.319171      380.635638       Champions
1183569707123        5.0    152.0  156.0           47.07                4.896924      230.221868  Need Attention
1183582322803        4.0    123.0  156.0           53.49                4.520851      242.910696  Need Attention
1183584551027        5.0    152.0  156.0           43.79                4.896924      214.464568     Hibernating
1183594184819        5.0    152.0  156.0           43.99                4.896924      215.425378     Hibernating

In [65]:
# Calculate the mean of each column within the 'clv' DataFrame grouped by 'Segment'.
clv.groupby('Segment').mean()


frequency     recency           T  monetary_value  six_Months_CLV
Segment                                                                           
Hibernating       3.750226  117.220814  155.078733       47.117612      211.100414
Need Attention    3.985441  125.137398  154.201092       54.114294      245.963066
Loyal Customers   3.351228  106.268426  136.388535       66.896035      300.817684
Champions         3.584015  117.536785  145.468665      146.735710      645.252504

In [70]:
# Calculate the mean of each column within the 'clv' DataFrame grouped by 'Segment'.
segment_means = clv.groupby('Segment').mean()

# count the number of customers in each 'Segment'.
segment_counts = clv.groupby('Segment').size()

# Add the customer count to the segment_means
segment_means['Customer_Count'] = segment_counts

print(segment_means)



                 frequency     recency           T  monetary_value  six_Months_CLV  Customer_Count
Segment                                                                                           
Hibernating       3.750226  117.220814  155.078733       47.117612      211.100414            1105
Need Attention    3.985441  125.137398  154.201092       54.114294      245.963066            1099
Loyal Customers   3.351228  106.268426  136.388535       66.896035      300.817684            1099
Champions         3.584015  117.536785  145.468665      146.735710      645.252504            1101


**Hibernating (1105 customers):** These customers have a lower interaction frequency and their last interaction was quite a while ago. They spend less on average per transaction and have a lower expected 6-month CLV, indicating they might be at risk of churning or may have already lapsed. Strategies may be needed to re-engage them.

**Need Attention (1099 customers):** Customers in this segment interact a bit more frequently and have spent more recently than the Hibernating segment. Their higher expected 6-month CLV suggests they have potential to contribute more if the company can successfully draw their attention, possibly through personalized marketing or loyalty incentives.

**Loyal Customers (1099 customers):** This segment has a similar size to the Need Attention group but exhibits a higher average spend and the highest expected 6-month CLV among the groups, indicating they are highly valuable. Maintaining their satisfaction and engagement is likely to be fruitful.

**Champions (1101 customers):** These customers do not have the highest frequency, but they spend the most on average and have a recent purchase history. Their expected 6-month CLV is significantly higher, indicating they are currently the most valuable and engaged. It's critical for the company to keep this group satisfied and to leverage their advocacy potential.

these insights could help in tailoring specific marketing strategies for each segment to maximize customer value. The Champions and Loyal Customers are essential for stable revenue, while the Need Attention and Hibernating segments represent opportunities for growth and re-engagement.






